##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# How to save and load models in NSL


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/neural-structured-learning/blob/master/neural_structured_learning/examples/notebooks/How_to_save_and_load_model_in_nsl.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/neural-structured-learning/blob/master/neural_structured_learning/examples/notebooks/How_to_save_and_load_model_in_nsl.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

In this example, we will demonstrate how to save and load `AdversarialRegularization` models in NSL. For more details about the concept
of adversarial regularization and how to use the `AdversarialRegularization`
API, please see this 
[tutorial](https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist)
or this [video explanation](https://youtu.be/Js2WJkhdU7k).

The high-level structure of this example is as follows:

1.  Create a neural network as a base model.
    * Here we create the base model with the `tf.keras` functional API.
    * This procedure is also compatible with models created by the
      `tf.keras.Sequential` API and models created by subclasses of
      `tf.keras.Model`, provided that the subclasses support saving and loading.
    * For more information on Keras models in TensorFlow, see this
      [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model).
2.  Wrap the base model with the **`AdversarialRegularization`** wrapper class.
    * The wrapper class is provided by the NSL framework, and it creates a new
      `tf.keras.Model` instance.
    * This new model will include the adversarial loss as a regularization term
      in its training objective.
3.  Train and evaluate the new model.
4.  Save the model, and load it back.
5.  Verify that the loaded model is the same as the one before saving.

## Setup

Install the Neural Structured Learning package.

In [ ]:
!pip uninstall -y neural-structured-learning

In [ ]:
!pip install --quiet neural-structured-learning

Import libraries. We abbreviate `neural_structured_learning` to `nsl`.

In [ ]:
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Hyperparameters

We collect and explain the hyperparameters (in an `HParams` object) for model
training and evaluation.

Input/Output:

*   **`input_shape`**: The shape of the input tensor. Each image is 28-by-28
pixels with 1 channel.
*   **`num_classes`**: There are a total of 10 classes, corresponding to 10
digits [0-9].

Model architecture:

*   **`conv_filters`**: A list of numbers, each specifying the number of
filters in a convolutional layer.
*   **`kernel_size`**: The size of 2D convolution window, shared by all
convolutional layers.
*   **`pool_size`**: Factors to downscale the image in each max-pooling layer.
*   **`num_fc_units`**: The number of units (i.e., width) of each
fully-connected layer.

Training and evaluation:

*  **`batch_size`**: Batch size used for training and evaluation.
*  **`epochs`**: The number of training epochs.

Adversarial learning:

*   **`adv_multiplier`**: The weight of adversarial loss in the training
objective, relative to the labeled loss.
*   **`adv_step_size`**: The magnitude of adversarial perturbation.
*  **`adv_grad_norm`**: The norm to measure the magnitude of adversarial
perturbation.


In [ ]:
class HParams(object):
  def __init__(self):
    self.input_shape = [28, 28, 1]
    self.num_classes = 10
    self.conv_filters = [32, 64, 64]
    self.kernel_size = (3, 3)
    self.pool_size = (2, 2)
    self.num_fc_units = [64]
    self.batch_size = 32
    self.epochs = 5
    self.adv_multiplier = 0.2
    self.adv_step_size = 0.2
    self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

## MNIST dataset

The [MNIST dataset](http://yann.lecun.com/exdb/mnist/) contains grayscale
images of handwritten digits (from '0' to '9'). Each image shows one digit at
low resolution (28-by-28 pixels). The task involved is to classify images into
10 categories, one per digit.

Here we load the MNIST dataset from
[TensorFlow Datasets](https://www.tensorflow.org/datasets). It handles
downloading the data and constructing a `tf.data.Dataset`. The loaded dataset
has two subsets:

*   `train` with 60,000 examples, and
*   `test` with 10,000 examples.

Examples in both subsets are stored in feature dictionaries with the following
two keys:

*   `image`: Array of pixel values, ranging from 0 to 255.
*   `label`: Groundtruth label, ranging from 0 to 9.

In [ ]:
datasets = tfds.load('mnist')

train_dataset = datasets['train']
test_dataset = datasets['test']

IMAGE_INPUT_NAME = 'image'
LABEL_INPUT_NAME = 'label'

To make the model numerically stable, we normalize the pixel values to [0, 1]
by mapping the dataset over the `normalize` function. After shuffling training
set and batching, we convert the examples to feature tuples `(image, label)`
for training the base model. We also provide a function to convert from tuples
to dictionaries for later use.

In [ ]:
def normalize(features):
  features[IMAGE_INPUT_NAME] = tf.cast(
      features[IMAGE_INPUT_NAME], dtype=tf.float32) / 255.0
  return features

def convert_to_tuples(features):
  return features[IMAGE_INPUT_NAME], features[LABEL_INPUT_NAME]

def convert_to_dictionaries(image, label):
  return {IMAGE_INPUT_NAME: image, LABEL_INPUT_NAME: label}

train_dataset = train_dataset.map(normalize).shuffle(10000).batch(HPARAMS.batch_size).map(convert_to_tuples)
test_dataset = test_dataset.map(normalize).batch(HPARAMS.batch_size).map(convert_to_tuples)

## Base model

Our base model will be a neural network consisting of 3 convolutional layers
follwed by 2 fully-connected layers (as defined in `HPARAMS`). Here we define
it using the Keras functional API. Feel free to try other APIs or model
architectures (e.g. subclassing). Note that the NSL framework does support all three types of Keras APIs.

In [ ]:
def build_base_model(hparams):
  """Builds a model according to the architecture defined in `hparams`."""
  inputs = tf.keras.Input(
      shape=hparams.input_shape, dtype=tf.float32, name=IMAGE_INPUT_NAME)

  x = inputs
  for i, num_filters in enumerate(hparams.conv_filters):
    x = tf.keras.layers.Conv2D(
        num_filters, hparams.kernel_size, activation='relu')(
            x)
    if i < len(hparams.conv_filters) - 1:
      # max pooling between convolutional layers
      x = tf.keras.layers.MaxPooling2D(hparams.pool_size)(x)
  x = tf.keras.layers.Flatten()(x)
  for num_units in hparams.num_fc_units:
    x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  pred = tf.keras.layers.Dense(hparams.num_classes)(x)
  model = tf.keras.Model(inputs=inputs, outputs=pred)
  return model

In [ ]:
base_model = build_base_model(HPARAMS)
base_model.summary()

## Adversarial-regularized model

Here we show how to incorporate adversarial training into a Keras model with a
few lines of code, using the NSL framework. The base model is wrapped to create
a new `tf.Keras.Model`, whose training objective includes adversarial
regularization.

First, we create a config object with all relevant hyperparameters using the
helper function `nsl.configs.make_adv_reg_config`.

In [ ]:
adv_config = nsl.configs.make_adv_reg_config(
    multiplier=HPARAMS.adv_multiplier,
    adv_step_size=HPARAMS.adv_step_size,
    adv_grad_norm=HPARAMS.adv_grad_norm
)

Now we can wrap a base model with `AdversarialRegularization`. Here we create a
new base model (`base_adv_model`), so that the existing one (`base_model`) can
be used in later comparison.

The returned `adv_model` is a `tf.keras.Model` object, whose training objective
includes a regularization term for the adversarial loss. To compute that loss,
the model has to have access to the label information (feature `label`), in
addition to regular input (feature `image`). For this reason, we convert the
examples in the datasets from tuples back to dictionaries. And we tell the
model which feature contains the label information via the `label_keys`
parameter.

In [ ]:
base_adv_model = build_base_model(HPARAMS)
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

train_set_for_adv_model = train_dataset.map(convert_to_dictionaries)
test_set_for_adv_model = test_dataset.map(convert_to_dictionaries)

In [ ]:
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

Next we compile, train, and evaluate the
adversarial-regularized model. There might be warnings like
"Output missing from loss dictionary," which is fine because
the `adv_model` doesn't rely on the base implementation to
calculate the total loss.

Along training, we save the model weights in a checkpoint after each epoch.
The checkpoint can be restored by using `adv_model.load_weights`.

In [ ]:
adv_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

adv_model.fit(
    train_set_for_adv_model,
    epochs=2,
    callbacks=[tf.keras.callbacks.ModelCheckpoint('ckpt')])

In [ ]:
results = adv_model.evaluate(test_set_for_adv_model)
for metric, value in zip(adv_model.metrics_names, results):
  print(metric, value)

## Save and Load

After training, we can save the model in a SavedModel format, which stores both
architecture and weights in the model. For `AdversarialRegularization` models,
the `save` method saves the base model because the `AdversarialRegularization`
wrapper doesn't change the architecture nor add any variables in model weights.

In [ ]:
adv_model.save('base_model') # only save base model

Now we can load the saved model using `tf.keras.models.load_model`. The model
summary shows the same architecture as we saw in the [Base model](#scrollTo=JrrMpPNmpCKK) section.

In [ ]:
loaded_base_model = tf.keras.models.load_model('base_model')
loaded_base_model.summary()

We can also verify that the model weights are the same.

In [ ]:
original_weights = adv_model.base_model.trainable_weights
loaded_weights = loaded_base_model.trainable_weights

all(np.array_equal(original_weight.numpy(), loaded_weight.numpy())
    for original_weight, loaded_weight in zip(original_weights, loaded_weights))

And so are the predictions and evaluation metrics.

In [ ]:
adv_base_preds = adv_model.base_model.predict(test_dataset)
load_base_preds = loaded_base_model.predict(test_dataset)

np.array_equal(adv_base_preds, load_base_preds)

In [ ]:
adv_model.base_model.evaluate(test_dataset)

In [ ]:
loaded_base_model.evaluate(test_dataset)

If you would like to continue adversarial training on the loaded model, you can
wrap it with `AdversarialRegularization` again.

In [ ]:
# We need to set the adv_config and other related parameters again.
adv_loaded_model = nsl.keras.AdversarialRegularization(
    loaded_base_model,
    label_keys=[LABEL_INPUT_NAME],
    adv_config=adv_config
)

In [ ]:
adv_loaded_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
adv_loaded_model.fit(
    train_set_for_adv_model,
    epochs=3,
    callbacks=[tf.keras.callbacks.ModelCheckpoint('ckpt')])

In [ ]:
results = adv_loaded_model.evaluate(test_set_for_adv_model)
for metric, value in zip(adv_loaded_model.metrics_names, results):
  print(metric, value)

You can see that the continued training improved the model.